<a href="https://colab.research.google.com/github/tada20001/NLP_2023/blob/main/CH18_08_BERT%EC%9D%98_%EB%AC%B8%EC%9E%A5_%EC%9E%84%EB%B2%A0%EB%94%A9(SBERT)%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%ED%95%9C%EA%B5%AD%EC%96%B4_%EC%B1%97%EB%B4%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SBERT를 이용하여 문장 임베딩을 얻을 수 있는 패키지인 sentence_transformers를 사용하여 쉽고 간단하게 한국어 챗봇을 구현함


In [1]:
pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 27.5 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=346d1a40e67033970daf2eed327b5ef7a3539a9c95a76d5fa8e68ae1ba0b58b2
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


### 1. 데이터 로드 및 전처리

In [2]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
from sentence_transformers import SentenceTransformer

In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/\
Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")

('ChatBotData.csv', <http.client.HTTPMessage at 0x7f2f3eb7b310>)

In [4]:
train_data = pd.read_csv('ChatBotData.csv')
train_data.tail()

,Q,A,label
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2
11822,힘들어서 결혼할까봐,도피성 결혼은 하지 않길 바라요.,2


### 2. 모델링

문장 임베딩을 얻기 위해 사전 훈련된 BERT를 로드함. 여기서는 한국어도 포함되어 학습된 다국어 모델을 로드함

In [6]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

이 모델은 100가지 언어를 지원하는 다국어 BERT BASE 모델로 SNLI 데이터를 학습한 후 STS-B 데이터로 학습되었으며, 문장 표현을 얻기 위해서는 평균 풀링(mean-tokens)을 사용했다는 의미임. 다시 말해 NLI데이터를 학습한 후에 STS 데이터로 추가 파인튜닝한 모델이라는 의미임...


SentenceTransformer로 로드할 수 있는 다양한 모델에 대한 리스트는 아래의 링크에서 확인 가능함

* 링크: https://huggingface.co/models?library=sentence‑transformers

데이터에서 모든 질문열, 즉 train_data['Q']에 대해 문장 임베딩 값을 구한 후 embedding이라는 새로운 열에 저장함

In [7]:
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis=1)

두개의 벡터로부터 코사인 유사도를 구하는 함수 cos_sim을 정의함

In [8]:
def cos_sim(A, B):
  return dot(A, B) / (norm(A)*norm(B))

return_answer함수는 임의의 질문이 들어오면 해당 질문의 문장 임베딩 값과 챗봇 데이터의 임베딩 열, 즉 train_data['embedding']에 저장해 둔 모든 질문 샘플들의 문장 임베딩 값들을 전부 비교하여 코사인 유사도 값이 가장 높은 질문 샘플을 찾아냄. 그리고 해당 질문 샘플과 짝이 되는 답변 샘플을 리턴함

In [11]:
def return_answer(question):
  embedding = model.encode(question)
  train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
  return train_data.loc[train_data['score'].idxmax()]['A']

In [12]:
return_answer('결혼하고 싶어')

'좋은 사람이랑 결혼할 수 있을 거예요.'

In [13]:
return_answer('나랑 커피먹을래?')

'카페인이 필요한 시간인가 봐요.'

In [14]:
return_answer('반가워')

'저도 반가워요.'

In [15]:
return_answer('잘가')

'안녕.'

In [16]:
return_answer('난 짜장면이 좋아')

'사랑하는 순간은 행복했잖아요.'

In [17]:
return_answer('뭔 소리야')

'의미 부여하지 마세요.'

In [18]:
return_answer('일하기 싫다')

'저도요!!'